In [90]:
from astropy.modeling import models, fitting
import matplotlib.pyplot as plt
import random
import re

In [91]:
two_gaussian = None

for i in range(3):
    m = random.randint(10, 490)
    amp = random.randint(100, 150)
    stdd = random.randint(5, 12)
    gaussian = models.Gaussian1D(mean=m, amplitude=amp, stddev=stdd) 
    
    if two_gaussian is None:
        two_gaussian = gaussian
    else:
        two_gaussian += gaussian

# two_gaussian = models.Gaussian1D(mean=158, amplitude=120, stddev=8) + \
#     models.Gaussian1D(mean=200, amplitude=80, stddev=10)

In [92]:
def get_maximum_nsigma(model):
    max_sigmas = []
    stddev_keys = [key for key in model._param_names if 'stddev' in key]

    mean_keys = [key for key in model._param_names if 'mean' in key]

    stddev_values = [
            model.__getattribute__(key).value for key in stddev_keys]

    mean_values = [
            model.__getattribute__(key).value for key in mean_keys]
    
    sorted_mean_keys = [x for (y,x) in sorted(zip(mean_values, mean_keys))]
    
    print(mean_values)
    print(sorted_mean_keys)
    for i in range(len(sorted_mean_keys)):
        for e in range(len(sorted_mean_keys)):
            if e != i:
                print(sorted_mean_keys[i], sorted_mean_keys[e])
        
        key_1 = sorted_mean_keys[i]
        key_2 = sorted_mean_keys[i + 1]
        
        stddev_1 = re.sub('mean', 'stddev', key_1)
        stddev_2 = re.sub('mean', 'stddev', key_2)
        
        mean_1 = model.__getattribute__(key_1).value
        mean_2 = model.__getattribute__(key_2).value
        
        # limit1 = model[i].mean.value
        limit_1 = model.__getattribute__(key_1).value
        limit_2 = model.__getattribute__(key_2).value
        print(limit_1, limit_2)
        
        nsigma = 0
        while limit_1 < limit_2:
            nsigma += 1
            limit_1 = mean_1 + nsigma * model[i].stddev.value
            limit_2 = mean_2 - nsigma * model[i + 1].stddev.value
            
            print(nsigma)
        else:
            max_sigmas.append(nsigma - 1)
    print(max_sigmas)
    return max_sigmas
    # print(mean_values)

In [93]:
max_sigmas = get_maximum_nsigma(model=two_gaussian)

for s in max_sigmas:
    print('Using {:d} sigmas to each side'.format(s))

x_axis = range(500)

plt.plot(two_gaussian(x_axis), color='b')
for sub_model in two_gaussian:
    plt.axvline(sub_model.mean.value, color='r')
    plt.axvspan(sub_model.mean.value - nsigma * sub_model.stddev.value,
                sub_model.mean.value + nsigma * sub_model.stddev.value,
                color='r',
                alpha=0.3)

plt.show()



[450.0, 447.0, 71.0]
[u'mean_2', u'mean_1', u'mean_0']
(u'mean_2', u'mean_1')
(u'mean_2', u'mean_0')
(71.0, 447.0)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
(u'mean_1', u'mean_2')
(u'mean_1', u'mean_0')
(447.0, 450.0)
1
(u'mean_0', u'mean_2')
(u'mean_0', u'mean_1')


IndexError: list index out of range